In [3]:
import keras
import numpy as np
import tensorflow as tf

Using TensorFlow backend


In [8]:
class OutputMultiplication(keras.layers.Layer):
    def __init__(self):
        super(OutputMultiplication,self).__init__(trainable=False)

    def call(self, arr1, arr2):
        return arr1 * arr2
    
# Ejemplo:
OutputMultiplication()(np.array([1.,0.,1.]), np.array([0.3783, 1.49331, 9.128]))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.3783, 0.    , 9.128 ], dtype=float32)>

In [5]:
class LayerExpandOutput(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(LayerExpandOutput, self).__init__(**kwargs, trainable=False)

    def call(self, inputs):
        dim = tf.shape(inputs)[1]
        expanded = tf.repeat(inputs, repeats= dim, axis= 1)

        return expanded
    
# Ejemplo:
#   Entrada: [1,0,1]
#   Salida: [1,1,1,0,0,0,1,1,1]
LayerExpandOutput()(np.array([[1,0,1],[2,0,3]]))

<tf.Tensor: shape=(2, 9), dtype=int32, numpy=
array([[1, 1, 1, 0, 0, 0, 1, 1, 1],
       [2, 2, 2, 0, 0, 0, 3, 3, 3]])>

In [4]:
import tensorflow as tf

# Datos iniciales
data = tf.constant([[[1, 2, 3], [1, 2, 4], [1, 2, 5]]], dtype=tf.float32)  # [batch_size, sequence_length, features]
labels = tf.constant([[1, 0, 1]], dtype=tf.float32)  # [batch_size, sequence_length]


tf.Tensor(
[[[1. 2. 3. 1.]
  [1. 2. 4. 0.]
  [1. 2. 5. 1.]]], shape=(1, 3, 4), dtype=float32)
tf.Tensor(
[[[1. 2. 3. 0. 0. 0.]
  [1. 2. 4. 0. 0. 0.]
  [1. 2. 5. 0. 0. 0.]]], shape=(1, 3, 6), dtype=float32)
tf.Tensor(
[[[1. 2. 3. 0.]
  [1. 2. 4. 1.]
  [1. 2. 5. 0.]]], shape=(1, 3, 4), dtype=float32)


In [18]:
import tensorflow as tf

# Datos originales
datos_originales = [[1, 2, 3], [4, 5, 6], [7,8,9]]  # Por ejemplo, 2 arreglos

# Convertir la lista de arreglos en un tensor
tensor_arreglos = tf.constant(datos_originales, dtype=tf.float32)

# Crear una matriz identidad de la misma forma que los arreglos
matriz_identidad = tf.eye(tensor_arreglos.shape[-1], dtype=tf.float32)

# Multiplicar cada arreglo por la matriz identidad para obtener la matriz diagonal
matrices_diagonales = tensor_arreglos[:, :, tf.newaxis] * matriz_identidad

# Imprimir el resultado
print(matrices_diagonales)

tf.Tensor(
[[[1. 0. 0.]
  [0. 2. 0.]
  [0. 0. 3.]]

 [[4. 0. 0.]
  [0. 5. 0.]
  [0. 0. 6.]]

 [[7. 0. 0.]
  [0. 8. 0.]
  [0. 0. 9.]]], shape=(3, 3, 3), dtype=float32)


In [9]:
import tensorflow as tf
from keras.layers import Concatenate, Input

class ConcatenationLayer(keras.layers.Layer):
    def __init__(self, **kwargs) -> None:
        super(ConcatenationLayer, self).__init__(**kwargs)

    def call(self, inputs: tf.TensorArray) -> tf.TensorArray:
        matrix = inputs
        labels = tf.ones(matrix.shape[1])
        labels = tf.expand_dims(labels, axis = 0)
        labels = tf.repeat(labels, repeats= tf.shape(matrix)[0], axis= 0)
        
        # Crear una matriz identidad de la misma forma que los arreglos
        matriz_identidad = tf.eye(labels.shape[-1], dtype=tf.float32)

        # Multiplicar cada arreglo por la matriz identidad para obtener la matriz diagonal
        matrices_diagonales = labels[:, :, tf.newaxis] * matriz_identidad

        test = tf.expand_dims(matrices_diagonales, axis= -1)

        matrices_copiadas = tf.expand_dims(matrix, axis= 1)
        matrices_copiadas = tf.repeat(matrices_copiadas, repeats= labels.shape[1], axis= 1)

        results = Concatenate(axis= 3)([matrices_copiadas, test])

        return results

    
# Ejemplo:
ConcatenationLayer()(tf.constant([[[1,2,5],[1,2,3],[1,2,4]], [[1,2,50],[1,2,30],[1,2,40]]], dtype= tf.float32))

<tf.Tensor: shape=(2, 3, 3, 4), dtype=float32, numpy=
array([[[[ 1.,  2.,  5.,  1.],
         [ 1.,  2.,  3.,  0.],
         [ 1.,  2.,  4.,  0.]],

        [[ 1.,  2.,  5.,  0.],
         [ 1.,  2.,  3.,  1.],
         [ 1.,  2.,  4.,  0.]],

        [[ 1.,  2.,  5.,  0.],
         [ 1.,  2.,  3.,  0.],
         [ 1.,  2.,  4.,  1.]]],


       [[[ 1.,  2., 50.,  1.],
         [ 1.,  2., 30.,  0.],
         [ 1.,  2., 40.,  0.]],

        [[ 1.,  2., 50.,  0.],
         [ 1.,  2., 30.,  1.],
         [ 1.,  2., 40.,  0.]],

        [[ 1.,  2., 50.,  0.],
         [ 1.,  2., 30.,  0.],
         [ 1.,  2., 40.,  1.]]]], dtype=float32)>

In [ ]:
[[1,2,50],[1,2,30],[1,2,40]]

In [ ]:
def create_model_CPMP_Nacho(source_model, destination_model, S, H):
    input = keras.layers.Input(shape= (S,H+1))
    output_source_model = LayerExpandOutput()(source_model(input))
    
    return model

In [ ]:
def create_model_CPMP_LosWillyAmigos(source_model, destination_model, S, H):
    input = keras.layers.Input(shape= (S,H+1))
    concat = ConcatenateLayerState()(input,source_model(input))
    output = destination_model(concat)
    model = keras.Model(inputs=input,outputs=output)

    for layer in model.layers:
        layer.trainable=False

    return model

In [50]:
from keras.layers import Concatenate

a = np.array([[1,2,3,4],[2,3,4,2],[1,1,1,1]])
b = np.array([[10], [5], [20]])

Concatenate()([a,b])

<tf.Tensor: shape=(3, 5), dtype=int32, numpy=
array([[ 1,  2,  3,  4, 10],
       [ 2,  3,  4,  2,  5],
       [ 1,  1,  1,  1, 20]])>